In [1]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import re

# 다나와 사이트 크롤링

### 크롬 드라이버 경로 지정

In [8]:
path = 'C:/Users/mbnb8/Downloads/chromedriver.exe'

In [9]:
driver = webdriver.Chrome(path) 
#여기서 우리가 제어하고자 하는 chrom 창이 뜸.
#그거 끄면 안됨

### 크롤링 하고자 하는 링크 get

In [10]:
driver.get('ㅋㅋ')
time.sleep(1)
#실행하면 해당 링크가 크롬창으로 열린다

### 1페이지 부터 여러페이지 끌어오기

In [175]:
text = []
user = []
recommendation = []
for i in range(1,10+1): 
    #1페이지에서 2페이지갈 때는 i:1, 2에서 3갈때는 i:2, 9에서 10갈때는 i:9
    text_page = driver.find_elements_by_class_name("comment")
    user_page = driver.find_elements_by_class_name("user")
    num_text_in_page = len(text_page)
    
    for j in range(num_text_in_page):
        text += [text_page[j].text]
        user += [user_page[j].text]
    
    for k in (2,4,6,8,10,12):
        recom_page = driver.find_elements_by_xpath("""//*[@id="qna_List"]/ul/li["""+str(k)+"""]/div/dl/dd/ul/li/div/a/strong""")
        recom_num = len(recom_page)
        recommendation_1 = []
        for a in range(recom_num):
            recommendation_1 += [recom_page[a].text]
        recommendation += [recommendation_1]
    
    driver.find_elements_by_xpath("""//*[@id="qna_List"]/ul/div/div/a["""+str(i)+"""]""")[0].click()
    time.sleep(1)

for h in range(3): #30으로 핳면 310페이지 까지
    for i in range(2,11+1):
        text_page = driver.find_elements_by_class_name("comment")
        user_page = driver.find_elements_by_class_name("user")
        num_text_in_page = len(text_page)
    
        for j in range(num_text_in_page):
            text += [text_page[j].text]
            user += [user_page[j].text]
            
        for k in (2,4,6,8,10,12):
            recom_page = driver.find_elements_by_xpath("""//*[@id="qna_List"]/ul/li["""+str(k)+"""]/div/dl/dd/ul/li/div/a/strong""")
            recom_num = len(recom_page)
            recommendation_1 = []
            for a in range(recom_num):
                recommendation_1 += [recom_page[a].text]
            recommendation += [recommendation_1]
    
        driver.find_elements_by_xpath("""//*[@id="qna_List"]/ul/div/div/a["""+str(i)+"""]""")[0].click()
        time.sleep(1)


In [12]:
data = pd.DataFrame({'user': user,'comment': text, 'recommendation': recommendation})

In [328]:
data.to_csv('data1_2.csv',encoding='ms949')

# 크롤링한 데이터 텍스트 처리

In [3]:
dat1 = pd.read_csv('data1_1.csv',encoding='ms949')
dat2 = pd.read_csv('data1_2.csv',encoding='ms949')

In [4]:
data = pd.concat([dat1,dat2])

### user 열에서 0000.00.00의 날짜 정보만 뽑아오기

In [13]:
data['date'] = data['user'].str.extract('(\d{4}\.\d{2}\.\d{2})')

In [14]:
data['comment'][0]

'1. 사용 용도 (사용하는 프로그램/게임명 기재)\n라이트룸, 포토샵, 프리미어프로, 에프터이팩트(사진, 영상작업 위주이고 게임 안합니다)\n2. 원하는 가격대\n150~200만원\n3. 크기와 무게 (선호하는 크기, 무게 / 휴대 여부)\n되도록이면 2kg이내였으면 좋겠고, 2kg이 넘어도 상관은 없음\n4. 운영체제 필요 여부 (ex. 필요 없음, 윈도우10 등)\n윈도우10\n5. 선호하는 제조사\n상관없음(AS 편한 제조사)\n6. 기피하는 제조사\nAS 불편한 제조사\n7. 그 외 조건 (저장장치/색상/특정 입출력단자 등)\n광시야각, SRGB100%이상, 램16G이상, SSD256G이상'

### comment 열에서 '1' '2' '3' ... 을 나눠서  해당하는 텍스트 가져오기

In [15]:
purpose = []
for com in data['comment']:
    a = com.partition('2.')[0] #'2.' 이전의 문자열 가져오기
    b = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',a) # 특수문자 제거
    c = re.sub('\r\n',' ',b) # \r\n 제거
    purpose += [c]

In [15]:
#reindex
data.reset_index(inplace=True)

In [16]:
purpose = []
price = []
size = []
OS = []
brand_prefer = []
brand_hater = []
etc = []
for com in data['comment']:
    a = com.partition('2.') 
    b = a[2].partition('3.') 
    c = b[2].partition('4.') 
    d = c[2].partition('5.') 
    e = d[2].partition('6.') 
    f = e[2].partition('7.') 
    
    #c = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',b) # 특수문자 제거
    #d = re.sub('\r\n',' ',b) # \r\n 제거
    purpose += [a[0]]
    price += [b[0]]
    size += [c[0]]
    OS += [d[0]]
    brand_prefer += [e[0]]
    brand_hater += [f[0]]
    etc += [f[2]]

In [17]:
purpose_new = []
price_new = []
size_new = []
OS_new = []
br_pre = []
br_hat = []
etcc = []
for pur,pr,sz,os,bp,bh,et in zip(purpose,price,size,OS,brand_prefer,brand_hater,etc):
    purpose_new += [pur.replace('1. ','').replace('사용 용도 (사용하는 프로그램/게임명 기재)','').replace('사용 용도','').replace('\r\n','')]
    price_new += [pr.replace('원하는 가격대','').replace('\r\n','')]
    size_new += [sz.replace('크기와 무게 (선호하는 크기, 무게 / 휴대 여부)','').replace('크기와 무게','').replace('\r\n','')]
    OS_new += [os.replace('운영체제 필요 여부 (ex. 필요 없음, 윈도우10 등)','').replace('운영체제 필요 여부','').replace('\r\n','')]
    br_pre += [bp.replace('선호하는 제조사','').replace('\r\n','')]
    br_hat += [bp.replace('기피하는 제조사','').replace('\r\n','')]
    etcc += [et.replace('그 외 조건 (저장장치/색상/특정 입출력단자 등)','').replace('그 외 조건','').replace('\r\n','')]

In [18]:
data3 = pd.DataFrame({'purpose': purpose_new,'price' : price_new,'size':size_new,'OS':OS_new,'brand_prefer':br_pre,'brande_hate':br_hat,'etc':etcc})

In [19]:
pd.concat([data,data3],1).to_csv('first_trial2.csv',encoding='ms949')

### word 단위로 쪼개기

In [16]:
purpose_detail = []
for pur in purpose:
    purpose_detail += [pur.split()]

purpose_df = pd.DataFrame({'purpose': purpose_detail})

data1 = pd.concat([data,purpose_df],axis=1)[['date','purpose','recommendation']]

In [17]:
data1.head()

,date,purpose,recommendation
0,2019.04.28,"[1, 사용, 용도, 사용하는, 프로그램, 게임명, 기재, 솔리드웍스, 게임은, 롤...",[]
1,2019.04.28,"[1, 사용, 용도, 사용하는, 프로그램, 게임명, 기재, 워드, 및, 사무용]",[]
2,2019.04.28,"[1, 사용, 용도, 사용하는, 프로그램, 게임명, 기재, 문서, 작업, 영화, 음...",[]
3,2019.04.28,"[1, 사용, 용도, 사용하는, 프로그램, 게임명, 기재, 강의들을때, 사용할, 자...",[]
4,2019.04.28,"[1, 사용, 용도, 사용하는, 프로그램, 게임명, 기재, 문서편집, 넷플릭스시청,...",[]


In [18]:
data1.to_csv('date & purpose split & recommendation.csv',encoding='ms949')